# Gen AI Tutorial - Code snippets


**Import OpenAI key**

Create a keys.py file  
Add an entry that looks like this in the file  
openai_api_key = "Your OPENAI API key goes here"


In [1]:
import os
from api_keys import openai_api_key
# , serpapi_api_key

os.environ["OPENAI_API_KEY"] = openai_api_key
# os.environ["SERPAPI_API_KEY"] = serpapi_api_key

**Install Python libraries**


In [ ]:
%pip install -qU langchain-core
%pip install -qU langchain-community
%pip install -qU langchain-openai
%pip install -qU langchain-google-genai
%pip install -qU langchain

%pip install -qU faiss-cpu
%pip install -qU hdbcli
%pip install -qU google-search-results
%pip install -qU numexpr
%pip install -qU pypdf

### Exercise 1:

Use ChatPromptTemplate to tell a joke about any topic


**Import required classes**


In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

**Initialize llm, create prompt, invoke**

System message sets the context and needs to be the first message - I guess you can skip it entirely as well  
Then you can have alternate human and ai messages - to maintain the conversation  

Note:  The output is an AIMessage


In [8]:
chat_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

output_parser = StrOutputParser()

chat_prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "You are a world famous comedian"),
        ("human", "Hi - what's up !"),
        ("ai", "Doing great...  And you ?"),
        ("human", "Meh...  Can you tell me a joke about {topic} to uplift my spirits")
    ])

chain = chat_prompt_template | chat_llm | output_parser | RunnableLambda(lambda x: x.upper())
chain.invoke({"topic": "cats"})


'ABSOLUTELY! WHY DID THE CAT SIT ON THE COMPUTER? \n\nBECAUSE IT WANTED TO KEEP AN EYE ON THE MOUSE! 🐱💻'

### Exercise 2:

Create a fictitious product with the following JSON structure - name, description, price, rating  
Output needs to be dict (not a string)

__Import required classes__

In [11]:
from langchain_core.output_parsers import JsonOutputParser
# from langchain_core.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field

__Create required structure, add it to chain__

Create class with required structure  
Initialize JsonOutputParser with the structure  

Note:  The output is a dict  

Bonus:  See if you can create 5 products...  Output needs to be a dict

In [ ]:
class Product(BaseModel):
    name: str = Field(description="Name of product")
    description: str = Field(description="Description of product")
    price: float = Field(description="Price of the product")
    rating: int = Field(description="Rating of the product from 1 to 5")

output_parser = JsonOutputParser(pydantic_object=Product)

chat_prompt_template = ChatPromptTemplate(
    messages = [
        ("system", "You are a helpful AI assistant.  \nFormatting Instructions: {format_instructions}"),
        ("human", "Can you create a fictitious product for the following category: {category}")
])
                                     

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({
    "category": "Electronics",
    "format_instructions": output_parser.get_format_instructions()
    })

# result = chain.invoke({
#     "category": "Electronics",
#     "format_instructions": output_parser.get_format_instructions()
#     })
# print(type(result))

In [ ]:
class Inventory(BaseModel):
    products: list[Product] = Field(description="This is the list of products")
    
output_parser = JsonOutputParser(pydantic_object=Inventory)

chat_prompt_template = ChatPromptTemplate(
    messages = [
        ("system", "You are a helpful AI assistant.  \nFormatting Instructions: {format_instructions}"),
        ("human", "Can you create 5 fictitious products for the following category: {category}")
])
                                     

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({
    "category": "Electronics",
    "format_instructions": output_parser.get_format_instructions()
    })

### Exercise 3:

Create a summary of a long text - assume the long text will exceed token size limit

__Import required classes__

In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain

__Load pdf file, use map_reduce method__

Read up on how map_reduce summarization works  
You can also have your own prompts - instead of using the built-in prompts like below

In [ ]:
loader = PyPDFLoader("my_man_jeeves_story1.pdf")
documents = loader.load()

summarize_chain = load_summarize_chain(llm=chat_llm, chain_type="map_reduce")

summary = summarize_chain.invoke(documents)
print(summary["output_text"])

### Exercise 4:

Implement RAG (use any vector DB of your choice) to query from some large pdf file

__Import required classes__

In [22]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser


__Create Embeddings, get relevant emebddings for query, invoke llm__

Create Embeddings using any vector database - we are using FAISS  
Search for relevant embeddings for your query  
Invoke llm with your relevant context and query


In [ ]:
embeddings_model = OpenAIEmbeddings()
output_parser = StrOutputParser()

loader = PyPDFLoader("my_man_jeeves_story1.pdf")
documents = loader.load()

db = FAISS.from_documents(documents, embeddings_model)

query = "Who is Jeeves ?"
docs = db.similarity_search(query, k=4)

context = "\n\n".join([doc.page_content for doc in docs])
prompt = f"Context:\n{context}\n\nQuestion: {query}"

chain = chat_llm | output_parser
chain.invoke(prompt)

### Exercise 5:

Use Agents to find out the number of Olympic medals won by United States (or any country) in 2024

__Import required classes__

In [24]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

__Load all the tools, initialize agent, submit prompt__

Load all the tools that you think you might need - Typically you will load more than 1 tool here  
Initialize the agent  
Submit the prompt

In [ ]:
toolkit = load_tools(["serpapi"], llm=chat_llm)
agent = initialize_agent(toolkit, chat_llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

response = agent({"input":"How many gold medals did US win in 2024 Olympics ?"})